In [2]:
from langchain_google_community import GmailToolkit

toolkit = GmailToolkit()

In [3]:
from langchain_google_community.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="token.json",
    scopes=["https://mail.google.com/"],
    client_secrets_file="credentials.json",
)
api_resource = build_resource_service(credentials=credentials)
toolkit = GmailToolkit(api_resource=api_resource)

In [4]:
tools = toolkit.get_tools()
tools

[GmailCreateDraft(api_resource=<googleapiclient.discovery.Resource object at 0x7f8ee984d340>),
 GmailSendMessage(api_resource=<googleapiclient.discovery.Resource object at 0x7f8ee984d340>),
 GmailSearch(api_resource=<googleapiclient.discovery.Resource object at 0x7f8ee984d340>),
 GmailGetMessage(api_resource=<googleapiclient.discovery.Resource object at 0x7f8ee984d340>),
 GmailGetThread(api_resource=<googleapiclient.discovery.Resource object at 0x7f8ee984d340>)]

In [5]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [6]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [8]:
example_query = "Draft an email to fake@fake.com thanking them for coffee."

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Draft an email to fake@fake.com thanking them for coffee.
================================== Ai Message ==================================
Tool Calls:
  create_gmail_draft (call_zQ8vS0VS3wOHPEN2vN3DLWCM)
 Call ID: call_zQ8vS0VS3wOHPEN2vN3DLWCM
  Args:
    message: Hi,

I just wanted to take a moment to thank you for the coffee. It was great to catch up and enjoy a nice cup together.

Looking forward to our next meeting!

Best regards,

[Your Name]
    to: ['fake@fake.com']
    subject: Thank You for the Coffee!
================================= Tool Message =================================
Name: create_gmail_draft

Draft created. Draft Id: r403262007375913709
================================== Ai Message ==================================

I have drafted an email to thank them for the coffee. If you need any changes or would like to send it, just let me know!


In [10]:
example_query = "Read the newest email I received and summarize its contents."

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Read the newest email I received and summarize its contents.
================================== Ai Message ==================================
Tool Calls:
  search_gmail (call_YeOq88q3nHqCQOHfsrAvYLAT)
 Call ID: call_YeOq88q3nHqCQOHfsrAvYLAT
  Args:
    query: is:inbox
    resource: messages
    max_results: 1
================================= Tool Message =================================
Name: search_gmail

[{"id": "19293e85418de915", "threadId": "19293e724859f2e6", "snippet": "Dear Jiaxin, I hope this message finds you well. I&#39;ve been following your work and I am genuinely impressed by your experience. I would love the opportunity to learn more about your experiences and", "body": "Dear Jiaxin,\r\n\r\nI hope this message finds you well. I’ve been following your work and I am\r\ngenuinely impressed by your experience. I would love the opportunity to\r\nlearn more about your experiences and insights.\

In [14]:
example_query = "Read my newest draft email in my gmail account and check if there's any grammar mistake."

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Read my newest draft email in my gmail account and check if there's any grammar mistake.
================================== Ai Message ==================================
Tool Calls:
  search_gmail (call_7Z6iA7YQx3FTCET2nuOLxDdH)
 Call ID: call_7Z6iA7YQx3FTCET2nuOLxDdH
  Args:
    query: is:draft
    resource: messages
    max_results: 1
================================= Tool Message =================================
Name: search_gmail

[{"id": "19293f384410c1b4", "threadId": "19293d8fbffdb5c0", "snippet": "Hi, I just wanted to take a moment to thank you for the coffee. It was great to catch up and enjoy a nice cup together. Looking forward to our next meeting! User Jiaxin&#39;s personal note: If you, the", "body": "Hi,\r\n\r\nI just wanted to take a moment to thank you for the coffee. It was great to \r\ncatch up and enjoy a nice cup together.\r\n\r\nLooking forward to our next meeting!\r\n\r\n\r\n\r\n\r\

In [7]:
example_query = "Summarise the five latest emails I received."

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Summarise the five latest emails I received.
================================== Ai Message ==================================
Tool Calls:
  search_gmail (call_Oa0VZGeRt1Cg32VeG6O0xgsz)
 Call ID: call_Oa0VZGeRt1Cg32VeG6O0xgsz
  Args:
    query: is:inbox
    resource: messages
    max_results: 5
================================= Tool Message =================================
Name: search_gmail

[{"id": "19293fc5f2347929", "threadId": "19293fc58628ac40", "snippet": "Address not found Your message wasn&#39;t delivered to recipient@example.com because the domain example.com couldn&#39;t be found. Check for typos or unnecessary spaces and try again. LEARN MORE This", "body": "\r\n** Address not found **\r\n\r\nYour message wasn't delivered to recipient@example.com because the domain example.com couldn't be found. Check for typos or unnecessary spaces and try again.\r\n\r\nLearn more here: https://www.rfc-editor